In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from hit_rate import HitRate

from pyspark.sql import functions as F
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[*]") \
                    .config('spark.ui.showConsoleProgress', 'false')\
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

22/10/06 02:24:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/06 02:24:43 WARN util.Utils: Service 'sparkDriver' could not bind on port 7080. Attempting port 7081.
22/10/06 02:24:45 WARN shortcircuit.DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.


<SparkContext master=local[*] appName=MovieRecomender>
Spark App Name : MovieRecomender


In [14]:
df = spark.read.option("sep", "::").csv("file:///data/zlb/zlb-account/hanh4/hanh4/data/ratings.dat")
df.na.drop()
df = df.toDF(*["UserID", "MovieID", "Rating", "Timestamp"])
df.createOrReplaceTempView("dataset");
df = df.cache()
df.count() #force cache

1000209

In [15]:
sql = '''
select 
  A.UserID, A.MovieID, Rating, ROW_NUMBER() OVER (partition by A.UserID order by Rating desc) as RowNumber
from 
  (
    select 
      * 
    from 
      (
        select 
          distinct(UserID) 
        from 
          dataset
      ), 
      (
        select 
          distinct(MovieID) 
        from 
          dataset
      )
  ) as A left outer join dataset as B
  on (A.UserID, A.MovieID) = (B.UserID, B.MovieID)
'''
full_matrix = spark.sql(sql)
full_matrix.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|RowNumber|
+------+-------+------+---------+
|  1090|   1088|  null|        1|
|  1090|   1139|  null|        2|
|  1090|    128|  null|        3|
|  1090|   1297|  null|        4|
|  1090|   1398|  null|        5|
|  1090|   1651|  null|        6|
|  1090|   1837|  null|        7|
|  1090|   1926|  null|        8|
|  1090|    199|  null|        9|
|  1090|   2291|  null|       10|
|  1090|   2711|  null|       11|
|  1090|   2777|  null|       12|
|  1090|   2817|  null|       13|
|  1090|   2876|  null|       14|
|  1090|   2953|  null|       15|
|  1090|   3074|  null|       16|
|  1090|    562|  null|       17|
|  1090|    657|  null|       18|
|  1090|    815|  null|       19|
|  1090|    898|  null|       20|
+------+-------+------+---------+
only showing top 20 rows



In [16]:
full_matrix.count()

22384240

Leave one out for each group in full_matrix

In [ ]:
loo_matrix = full_matrix
loo_train_matrix = full_matrix.filter(F.col("RowNumber") == 1)
loo_test_matrix = full_matrix.filter(F.col("RowNumber") != 1)

loo_train_matrix.show()
loo_test_matrix.show()

In [ ]:
(train, test) = df.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False,coldStartStrategy="drop")

In [ ]:
grid_search = ParamGridBuilder().addGrid(als.rank,[30,50]).addGrid(als.maxIter,[15,20,50]).addGrid(als.regParam, [0.05] ).build()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 

In [ ]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=evaluator, numFolds=5)

In [ ]:
spark.sparkContext.setCheckpointDir('checkpoint/')
cv_fitted=cv.fit(train)

In [ ]:
print(cv_fitted.bestModel.rank, cv_fitted.bestModel._java_obj.parent().getMaxIter(),cv_fitted.bestModel._java_obj.parent().getRegParam())

In [ ]:
evaluator.evaluate(cv_fitted.transform(test).na.drop())

In [ ]:
hr_evaluator = HitRate(predictionCol='prediction', labelCol='rating', userCol='userId')
model = als.fit(train)
predictions = model.transform(test)
hr_evaluator.evaluate(predictions)